In [ ]:
import subprocess
from datetime import datetime
from pathlib import Path
from typing import Literal

import fsspec
from utils import sync_notebook

In [ ]:
fs = fsspec.filesystem("file")
current_date = datetime.today().strftime("%Y-%m-%d")
output_folder = f"results/{current_date}/"
fs.mkdirs(output_folder, exist_ok=True)

In [ ]:
dataset = "mursst"

In [ ]:
def run_pyinstrument(file: str, output_format: Literal["json", "html"]):
    output_file = (
        output_folder
        + "pyinstrument-"
        + dataset
        + "-"
        + Path(file).stem
        + "."
        + output_format
    )
    command = [
        "pyinstrument",
        "--outfile",
        output_file,
        "-r",
        output_format,
        file,
        "--dataset",
        dataset,
    ]
    subprocess.run(command)

In [ ]:
if dataset == "gpm_imerg":
    input_methods = ["rioxarray", "odc", "pyresample", "rioxarray", "xesmf"]
elif dataset == "mursst":
    input_methods = ["odc", "pyresample", "rioxarray", "rasterio"]

In [ ]:
notebooks = []
for fp in input_methods:
    notebooks.extend(fs.glob(f"resample-{fp}*.ipynb"))

In [ ]:
for file in notebooks:
    sync_notebook(file)

In [ ]:
modules = []
for fp in input_methods:
    modules.extend(fs.glob(f"resample-{fp}*.py"))
for file in modules:
    # Skip kerchunk since it requires a different image build due to incompatibility with Zarr V3
    if "kerchunk" not in file:
        run_pyinstrument(file, "json")

In [ ]:
modules